# 🛍️ ShopSense — Phase 9: FastAPI + Gradio + Deployment
**Environment:** VS Code (local)

### What this phase does:
1. Sets up the project structure for deployment
2. Tests FastAPI backend locally
3. Tests Gradio UI locally
4. Deploys to Hugging Face Spaces (free public hosting)

### Files created:
```
ShopSense/
├── app.py              ← FastAPI backend
├── gradio_app.py       ← Gradio frontend (main app for HF Spaces)
├── requirements.txt    ← dependencies for HF Spaces
└── deployment.ipynb    ← this notebook
```

### Deployment target:
Hugging Face Spaces — free public hosting at:
`https://huggingface.co/spaces/YOUR_USERNAME/shopsense`

## Step 0 — Install Dependencies

In [1]:
import sys
!{sys.executable} -m pip install gradio fastapi uvicorn sentence-transformers faiss-cpu groq python-dotenv huggingface_hub -q
print('✅ Dependencies installed.')


[notice] A new release of pip is available: 25.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
✅ Dependencies installed.


## Step 1 — Verify All Required Files Exist

In [2]:
from pathlib import Path

required_files = [
    'data/products_clean.csv',
    'data/faiss_index.bin',
    'data/faiss_metadata.json',
    'data/reviews_faiss_index.bin',
    'data/reviews_metadata.json',
    'app.py',
    'gradio_app.py',
    'requirements.txt',
    '.env'
]

all_present = True
for f in required_files:
    exists = Path(f).exists()
    status = '✅' if exists else '❌ MISSING'
    print(f'  {status}  {f}')
    if not exists:
        all_present = False

print()
if all_present:
    print('✅ All files present — ready to test')
else:
    print('❌ Some files missing — check above before continuing')

  ✅  data/products_clean.csv
  ✅  data/faiss_index.bin
  ✅  data/faiss_metadata.json
  ✅  data/reviews_faiss_index.bin
  ✅  data/reviews_metadata.json
  ✅  app.py
  ✅  gradio_app.py
  ✅  requirements.txt
  ✅  .env

✅ All files present — ready to test


## Step 2 — Test FastAPI Backend Locally

Starts the FastAPI server in the background and runs a test request.
The server runs on http://localhost:8000

In [3]:
import subprocess
import time
import requests

# Start FastAPI server in background
print('Starting FastAPI server...')
server = subprocess.Popen(
    ['python', '-m', 'uvicorn', 'app:app', '--host', '0.0.0.0', '--port', '8000'],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# Wait for server to start
time.sleep(8)

# Test health check
try:
    r = requests.get('http://localhost:8000/')
    print(f'✅ Health check: {r.json()}')
except Exception as e:
    print(f'❌ Server not responding: {e}')
    print('   Try running: uvicorn app:app --reload  in your terminal')

Starting FastAPI server...
✅ Health check: {'status': 'ok', 'service': 'ShopSense API', 'version': '1.0.0', 'products_indexed': 10000}


In [4]:
# Test search endpoint
try:
    r = requests.post('http://localhost:8000/search', json={
        'query'          : 'non stick frying pan under $30',
        'include_reviews': False,
        'top_k'          : 3
    })
    data = r.json()
    print(f'✅ Search endpoint working')
    print(f'   Query refined : {data.get("query_refined")}')
    print(f'   Filters       : {data.get("filters_applied")}')
    print(f'   Results count : {len(data.get("results", []))}')
    print(f'   Explanation   : {data.get("explanation", "")[:80]}...')
    print()
    print('Top 3 results:')
    for i, r in enumerate(data.get('results', [])[:3], 1):
        print(f'  {i}. {r["title"][:60]} — ${r["price"]:.2f}')
except Exception as e:
    print(f'❌ Search failed: {e}')

✅ Search endpoint working
   Query refined : non stick frying pan
   Filters       : {'max_price': 30}
   Results count : 3
   Explanation   : Based on your search for a non-stick frying pan under $30, we've curated a list ...

Top 3 results:
  1. Anyfish 8 Inch Frying Pan without Lid Nonstick Induction Ski — $22.99
  2. MICHELANGELO Frying Pan with Lid, Nonstick 8 Inch Frying Pan — $23.99
  3. NGORAY Pancake Egg Frying Pan Nonstick Mini - 4 Cups Cast Ir — $22.99


In [5]:
# Test feedback endpoint
try:
    r = requests.post('http://localhost:8000/feedback', json={
        'asin'    : 'B0B12JWHF2',
        'title'   : 'Anyfish 8 Inch Frying Pan',
        'price'   : 22.99,
        'feedback': 'like'
    })
    print(f'✅ Feedback endpoint: {r.json()}')

    # Check session updated
    s = requests.get('http://localhost:8000/session').json()
    print(f'   Session liked count: {s["liked_count"]}')
except Exception as e:
    print(f'❌ Feedback failed: {e}')

# Shut down test server
server.terminate()
print('\n✅ FastAPI tests complete — server stopped')

✅ Feedback endpoint: {'status': 'liked', 'asin': 'B0B12JWHF2'}
   Session liked count: 1

✅ FastAPI tests complete — server stopped


## Step 3 — Test Gradio UI Locally

Launches the Gradio app locally so you can try it in your browser
before deploying to Hugging Face Spaces.

**Run this cell then open http://localhost:7860 in your browser.**
Press the Stop button in the notebook toolbar to shut it down when done.

In [6]:
# This launches the Gradio app
# Open http://localhost:7860 in your browser to test it
# Interrupt the kernel (■ button) to stop

import subprocess
print('Launching Gradio UI at http://localhost:7860')
print('Press the ■ Stop button in the toolbar to shut it down\n')

!python gradio_app.py

Launching Gradio UI at http://localhost:7860
Press the ■ Stop button in the toolbar to shut it down

Loading ShopSense...
Loading weights: 100%|█| 103/103 [00:00<00:00, 5833.13it/s, Materializing param=
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
✅ Ready
/Users/rithikabaskaran/MyFiles/ShopSense/gradio_app.py:342: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
^C
Keyboard interruption in main thread... closing server.


## Step 4 — Prepare for Hugging Face Spaces Deployment

HF Spaces needs:
1. `gradio_app.py` renamed to `app.py` OR a `README.md` that specifies the entry point
2. `requirements.txt`
3. All data files uploaded
4. Groq API key set as a Space Secret

We create a dedicated `YOUR_HF_TOKEN_HERE/` folder with everything organized.

In [7]:
import shutil
from pathlib import Path

HF_DIR = Path('YOUR_HF_TOKEN_HERE')
HF_DIR.mkdir(exist_ok=True)

# Copy app files
shutil.copy('gradio_app.py', HF_DIR / 'app.py')   # HF Spaces expects app.py
shutil.copy('requirements.txt', HF_DIR / 'requirements.txt')

# Copy data files
(HF_DIR / 'data').mkdir(exist_ok=True)
data_files = [
    'products_clean.csv',
    'faiss_index.bin',
    'faiss_metadata.json',
    'reviews_faiss_index.bin',
    'reviews_metadata.json'
]
for f in data_files:
    src = Path('data') / f
    if src.exists():
        shutil.copy(src, HF_DIR / 'data' / f)
        size = (HF_DIR / 'data' / f).stat().st_size / (1024*1024)
        print(f'  ✅ Copied data/{f} ({size:.1f} MB)')
    else:
        print(f'  ❌ MISSING: data/{f}')

print(f'\n✅ YOUR_HF_TOKEN_HERE/ folder ready')
print(f'   Total size: {sum(f.stat().st_size for f in HF_DIR.rglob("*") if f.is_file()) / (1024*1024):.1f} MB')

  ✅ Copied data/products_clean.csv (12.1 MB)
  ✅ Copied data/faiss_index.bin (14.6 MB)
  ✅ Copied data/faiss_metadata.json (13.0 MB)
  ✅ Copied data/reviews_faiss_index.bin (4.4 MB)
  ✅ Copied data/reviews_metadata.json (0.7 MB)

✅ YOUR_HF_TOKEN_HERE/ folder ready
   Total size: 44.9 MB


In [13]:
# Create HF Spaces README with metadata
# This tells HF Spaces how to run the app

readme_content = '''---
title: ShopSense
emoji: 🛍️
colorFrom: green
colorTo: green
sdk: gradio
sdk_version: 4.44.0
app_file: app.py
pinned: false
---

# 🛍️ ShopSense

AI-powered product recommendation system built with:
- **Semantic Search** — FAISS + sentence-transformers
- **Reranking** — cross-encoder (ms-marco-MiniLM)
- **RAG Review Summaries** — retrieve relevant reviews → Llama 3 summarization
- **LangGraph Agent** — intent extraction, filter parsing, explanation generation
- **Session Personalization** — remembers likes, dismissals, price preferences

## How to use
1. Type a natural language product query
2. Optionally set price/rating filters
3. Give 👍/👎 feedback to personalize future results
4. Check the Session Memory tab to see what the agent has learned

## Dataset
Amazon Reviews 2023 — Home & Kitchen category  
10,000 products · 5,566 reviews

## Tech Stack
Gradio · FastAPI · FAISS · sentence-transformers · LangGraph · Groq (Llama 3) · LoRA fine-tuning
'''

with open(HF_DIR / 'README.md', 'w') as f:
    f.write(readme_content)

print('✅ README.md created')

✅ README.md created


## Step 5 — Deploy to Hugging Face Spaces

### Method: HuggingFace Hub Python API

This uploads everything to a new Space automatically.
Make sure you have your HF token ready from https://huggingface.co/settings/tokens

In [17]:
from huggingface_hub import HfApi, create_repo

# ── Configuration ──────────────────────────────────────────────
HF_TOKEN    = 'YOUR_HF_TOKEN_HERE'   # from https://huggingface.co/settings/tokens
HF_USERNAME = 'shopsense_token'     # your HF username
SPACE_NAME  = 'Shopsense'
REPO_ID     = f'RithikaBaskaran/shopsense'

api = HfApi(token=HF_TOKEN)

# Create the Space (if it doesn't exist)
try:
    create_repo(
        repo_id   = REPO_ID,
        repo_type = 'space',
        space_sdk = 'gradio',
        token     = HF_TOKEN,
        exist_ok  = True
    )
    print(f'✅ Space created/verified: {REPO_ID}')
except Exception as e:
    print(f'Space creation note: {e}')

# Upload all files
print(f'\nUploading files to {REPO_ID}...')
print('This may take a few minutes for the data files...\n')

api.upload_folder(
    folder_path = str(HF_DIR),
    repo_id     = REPO_ID,
    repo_type   = 'space',
    token       = HF_TOKEN
)

print(f'\n✅ Deployment complete!')
print(f'   Your app: https://huggingface.co/spaces/{REPO_ID}')
print(f'\n⚠️  IMPORTANT: Set your Groq API key as a Space Secret:')
print(f'   1. Go to https://huggingface.co/spaces/{REPO_ID}/settings')
print(f'   2. Scroll to "Repository secrets"')
print(f'   3. Add: Name=GROQ_API_KEY  Value=your_actual_key')

✅ Space created/verified: RithikaBaskaran/shopsense

Uploading files to RithikaBaskaran/shopsense...
This may take a few minutes for the data files...



Processing Files (4 / 4): 100%|██████████| 46.3MB / 46.3MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
No files have been modified since last commit. Skipping to prevent empty commit.



✅ Deployment complete!
   Your app: https://huggingface.co/spaces/RithikaBaskaran/shopsense

⚠️  IMPORTANT: Set your Groq API key as a Space Secret:
   1. Go to https://huggingface.co/spaces/RithikaBaskaran/shopsense/settings
   2. Scroll to "Repository secrets"
   3. Add: Name=GROQ_API_KEY  Value=your_actual_key


In [22]:
from huggingface_hub import HfApi

api = HfApi(token=HF_TOKEN)

api.upload_file(
    path_or_fileobj = 'YOUR_HF_TOKEN_HERE/requirements.txt',
    path_in_repo    = 'requirements.txt',
    repo_id         = REPO_ID,
    repo_type       = 'space'
)
print('✅ requirements.txt updated — Space will rebuild automatically')

No files have been modified since last commit. Skipping to prevent empty commit.


✅ requirements.txt updated — Space will rebuild automatically


## Step 6 — Set Groq API Key as Space Secret

**CRITICAL — do this before testing the live app.**

The Groq API key must be set as a Space Secret (not in code) so it:
1. Never gets exposed in your public repository
2. Is available to the app as an environment variable

Steps:
1. Go to `https://huggingface.co/spaces/YOUR_USERNAME/shopsense/settings`
2. Scroll down to **"Repository secrets"**
3. Click **"New secret"**
4. Name: `GROQ_API_KEY`
5. Value: your actual `gsk_...` key
6. Click **"Add secret"**
7. The Space will automatically restart with the secret available

Once the Space shows **"Running"** status, your app is live!

## Step 7 — Verify Live Deployment

In [21]:
import requests
import time

SPACE_URL = f'https://{HF_USERNAME}-{SPACE_NAME}.hf.space'

print(f'Checking live app at {SPACE_URL}...')
print('(Space may take 2-5 min to build on first deploy)')

for attempt in range(5):
    try:
        r = requests.get(SPACE_URL, timeout=15)
        if r.status_code == 200:
            print(f'\n✅ App is live at {SPACE_URL}')
            break
        else:
            print(f'  Attempt {attempt+1}: status {r.status_code} — waiting...')
    except Exception as e:
        print(f'  Attempt {attempt+1}: not ready yet — waiting 30s...')
    time.sleep(30)
else:
    print(f'\nApp not responding yet — check build logs at:')
    print(f'https://huggingface.co/spaces/{REPO_ID}')

Checking live app at https://shopsense_token-Shopsense.hf.space...
(Space may take 2-5 min to build on first deploy)
  Attempt 1: not ready yet — waiting 30s...
  Attempt 2: not ready yet — waiting 30s...
  Attempt 3: not ready yet — waiting 30s...
  Attempt 4: not ready yet — waiting 30s...
  Attempt 5: not ready yet — waiting 30s...

App not responding yet — check build logs at:
https://huggingface.co/spaces/RithikaBaskaran/shopsense


## Step 8 — Final Summary

In [16]:
print('=' * 60)
print('     PHASE 9 COMPLETE — SHOPSENSE IS LIVE!')
print('=' * 60)
print()
print('  Live URL:')
print(f'    https://huggingface.co/spaces/{HF_USERNAME}/{SPACE_NAME}')
print()
print('  API endpoints (FastAPI):')
print('    GET  /           — health check')
print('    POST /search     — main search')
print('    POST /feedback   — like/dismiss')
print('    GET  /session    — session summary')
print('    POST /session/reset — reset memory')
print()
print('  UI features (Gradio):')
print('    🔍 Search with natural language queries')
print('    ⚙️  Price and rating filters')
print('    📝 RAG review summaries')
print('    👍 Explicit like/dismiss feedback')
print('    🧠 Session memory display')
print('    🔄 Session reset')
print()
print('  Full pipeline:')
print('    Phase 1 : Data preparation')
print('    Phase 2 : FAISS semantic retrieval')
print('    Phase 3 : Cross-encoder reranking')
print('    Phase 4 : RAG review summarization')
print('    Phase 5 : LangGraph agent')
print('    Phase 6 : Session personalization')
print('    Phase 7 : Evaluation (NDCG +9.7%)')
print('    Phase 8 : LoRA fine-tuning')
print('    Phase 9 : Deployment ← you are here')
print()
print('  🎉 ShopSense is complete!')
print('=' * 60)

     PHASE 9 COMPLETE — SHOPSENSE IS LIVE!

  Live URL:
    https://huggingface.co/spaces/shopsense_token/Shopsense

  API endpoints (FastAPI):
    GET  /           — health check
    POST /search     — main search
    POST /feedback   — like/dismiss
    GET  /session    — session summary
    POST /session/reset — reset memory

  UI features (Gradio):
    🔍 Search with natural language queries
    ⚙️  Price and rating filters
    📝 RAG review summaries
    👍 Explicit like/dismiss feedback
    🧠 Session memory display
    🔄 Session reset

  Full pipeline:
    Phase 1 : Data preparation
    Phase 2 : FAISS semantic retrieval
    Phase 3 : Cross-encoder reranking
    Phase 4 : RAG review summarization
    Phase 5 : LangGraph agent
    Phase 6 : Session personalization
    Phase 7 : Evaluation (NDCG +9.7%)
    Phase 8 : LoRA fine-tuning
    Phase 9 : Deployment ← you are here

  🎉 ShopSense is complete!


## Final GitHub Push
```bash
git add app.py gradio_app.py requirements.txt deployment.ipynb
git commit -m "phase 9: FastAPI + Gradio + HF Spaces deployment complete"
git push
```

> Make sure `YOUR_HF_TOKEN_HERE/` is in your `.gitignore` — it contains copies of data files.
> Also make sure your HF token is NOT in any committed file.